# MAK DRAFT HELPER 2019

In [612]:
import pandas as pd

## Batters

In [613]:
batters = pd.read_csv('./FantasyPros_2019_Projections_H.csv')

Fantasy Pros (fantasypros.com) produces an aggregated expert projections for each player across several key stats. No need to reinvent the wheel like Nate Silver did in his book with inefficient models - we just take what the experts project (using their models) and look at their consensus.

In [614]:
batters.head()

,Player,Team,Positions,AB,R,HR,RBI,SB,AVG,OBP,H,2B,3B,BB,SO,SLG,OPS,Own
0,Mike Trout,LAA,"CF,DH",509.0,113.0,39.0,97.0,24.0,0.307,0.440,156.0,29.0,4.0,121.0,128.0,0.613,1.053,99%
1,Mookie Betts,BOS,"CF,RF",591.0,117.0,30.0,93.0,27.0,0.307,0.387,182.0,43.0,4.0,77.0,94.0,0.547,0.934,99%
2,Jose Ramirez,CLE,"2B,3B",571.0,101.0,30.0,96.0,25.0,0.287,0.376,164.0,41.0,4.0,81.0,76.0,0.536,0.911,99%
3,Nolan Arenado,COL,3B,588.0,99.0,37.0,113.0,2.0,0.292,0.363,172.0,38.0,4.0,65.0,115.0,0.560,0.923,99%
4,J.D. Martinez,BOS,"LF,RF,DH",547.0,98.0,40.0,116.0,4.0,0.305,0.378,167.0,34.0,2.0,64.0,149.0,0.591,0.969,99%


In [615]:
del batters['OBP']
del batters['2B']
del batters['3B']
del batters['SLG']
del batters['OPS']

In [616]:
batters = batters[batters['AB'] >= 200]

In [617]:
batters['BB/KO'] = batters['BB'] / batters['SO']

For BB/KO, we want to look at the number of walks ABOVE the average BB/KO of the selection set (which is all batters projected to have over 200 AB's). This will approproately weight the average based on projected Fantasy production. We do the same with batting average.

In [618]:
bbko_mean = batters['BB/KO'].mean()
bbko_mean

0.3966791453006135

In [619]:
batters['BB+'] = (batters['BB/KO'] - bbko_mean) * batters['SO']

In [620]:
batters['AVG'] = batters['H'] / batters['AB']

In [621]:
avg_mean = batters['AVG'].mean()
avg_mean

0.2531137703777804

In [622]:
batters['AVG+'] = (batters['AVG'] - avg_mean) * batters['AB']

For this year, I'm simply standardizing each metric to the 0-100 linear scale with 100 being the top performer for that metric and 0 being the bottom. Since the scales are different for each one, this makes sense. In the past, I've used Z scores, but it is easier to be able to mentally calculate the actual metric in some cases. It also makes sense since these stats all accumulate over the course of the season/week.

In [623]:
batters['AVGS'] = round( ( batters['AVG+'] - batters['AVG+'].min() ) / ( batters['AVG+'].max() - batters['AVG+'].min() ) * 100 )
batters['BBKOS'] = round( ( batters['BB+'] - batters['BB+'].min() ) / ( batters['BB+'].max() - batters['BB+'].min() ) * 100 )
batters['HRS'] = round( ( batters['HR'] - batters['HR'].min() ) / ( batters['HR'].max() - batters['HR'].min() ) * 100 )
batters['RS'] = round( ( batters['R'] - batters['R'].min() ) / ( batters['R'].max() - batters['R'].min() ) * 100 )
batters['RBIS'] = round( ( batters['RBI'] - batters['RBI'].min() ) / ( batters['RBI'].max() - batters['RBI'].min() ) * 100 )
batters['SBS'] = round( ( batters['SB'] - batters['SB'].min() ) / ( batters['SB'].max() - batters['SB'].min() ) * 100 )
batters['SCORE'] = batters['AVGS'] + batters['BBKOS'] + batters['HRS'] + batters['RS'] + batters['RBIS'] + batters['SBS']
batters['RANK'] = batters['SCORE'].rank(method='average', na_option='keep', ascending=False, pct=False)

In [624]:
batters.sort_values(['SCORE'], ascending=[0])[0:5]

,Player,Team,Positions,AB,R,HR,RBI,SB,AVG,H,...,BB+,AVG+,AVGS,BBKOS,HRS,RS,RBIS,SBS,SCORE,RANK
0,Mike Trout,LAA,"CF,DH",509.0,113.0,39.0,97.0,24.0,0.306483,156.0,...,70.225069,27.165091,85.0,100.0,90.0,96.0,81.0,52.0,504.0,1.0
1,Mookie Betts,BOS,"CF,RF",591.0,117.0,30.0,93.0,27.0,0.307953,182.0,...,39.712160,32.409762,94.0,77.0,68.0,100.0,77.0,59.0,475.0,2.0
2,Jose Ramirez,CLE,"2B,3B",571.0,101.0,30.0,96.0,25.0,0.287215,164.0,...,50.852385,19.472037,72.0,85.0,68.0,84.0,80.0,54.0,443.0,3.0
4,J.D. Martinez,BOS,"LF,RF,DH",547.0,98.0,40.0,116.0,4.0,0.305302,167.0,...,4.894807,28.546768,87.0,50.0,93.0,81.0,100.0,9.0,420.0,4.0
3,Nolan Arenado,COL,3B,588.0,99.0,37.0,113.0,2.0,0.292517,172.0,...,19.381898,23.169103,79.0,61.0,85.0,82.0,97.0,4.0,408.0,5.5


## Pitchers

In [625]:
pitchers = pd.read_csv('./FantasyPros_2019_Projections_P.csv')

In [626]:
pitchers.head()

,Player,Team,Positions,IP,K,W,SV,ERA,WHIP,ER,H,BB,HR,G,GS,L,CG,Own
0,Max Scherzer,WSH,SP,207.6,269.0,16.0,0.0,2.90,1.00,67.0,154.0,54.0,26.0,32.0,32.0,8.0,2.0,99%
1,Chris Sale,BOS,SP,184.6,248.0,15.0,0.0,2.61,0.97,54.0,139.0,40.0,18.0,29.0,29.0,6.0,0.0,99%
2,Jacob deGrom,NYM,SP,207.8,248.0,15.0,0.0,2.72,1.05,63.0,167.0,51.0,19.0,32.0,32.0,8.0,1.0,99%
3,Justin Verlander,HOU,SP,200.8,244.0,15.0,0.0,3.15,1.05,70.0,160.0,50.0,27.0,32.0,31.0,8.0,1.0,99%
4,Corey Kluber,CLE,SP,207.1,220.0,16.0,0.0,3.22,1.07,74.0,180.0,42.0,24.0,32.0,32.0,8.0,3.0,99%


In [627]:
del pitchers['HR']
del pitchers['G']
del pitchers['GS']
del pitchers['CG']

In [628]:
pitchers = pitchers[pitchers['IP'] >= 40]

In [629]:
pitchers['ERA'] = pitchers['ER'] / pitchers['IP'] * 9
era_mean = pitchers['ERA'].mean()
pitchers['ERA+'] = ( era_mean - pitchers['ERA'] ) * pitchers['IP']

In [630]:
pitchers['WHIP'] = (pitchers['BB'] + pitchers['H']) / pitchers['IP']
whip_mean = pitchers['WHIP'].mean()
pitchers['WHIP+'] = ( whip_mean -  pitchers['WHIP'] ) * pitchers['IP']

In [631]:
pitchers['W-L'] = pitchers['W'] - pitchers['L']

In [632]:
pitchers['ERAS'] = round( ( pitchers['ERA+'] - pitchers['ERA+'].min() ) / ( pitchers['ERA+'].max() - pitchers['ERA+'].min() ) * 100 )
pitchers['WHIPS'] = round( ( pitchers['WHIP+'] - pitchers['WHIP+'].min() ) / ( pitchers['WHIP+'].max() - pitchers['WHIP+'].min() ) * 100 )
pitchers['KS'] = round( ( pitchers['K'] - pitchers['K'].min() ) / ( pitchers['K'].max() - pitchers['K'].min() ) * 100 )
pitchers['SVS'] = round( ( pitchers['SV'] - pitchers['SV'].min() ) / ( pitchers['SV'].max() - pitchers['SV'].min() ) * 100 )
pitchers['WLS'] = round( ( pitchers['W-L'] - pitchers['W-L'].min() ) / ( pitchers['W-L'].max() - pitchers['W-L'].min() ) * 100 )
pitchers['SCORE'] = pitchers['ERAS'] + pitchers['WHIPS'] + pitchers['KS'] + pitchers['SVS'] + pitchers['WLS']
pitchers['RANK'] = pitchers['SCORE'].rank(method='average', na_option='keep', ascending=False, pct=False)

In [633]:
pitchers.sort_values(['SCORE'], ascending=[0])[0:5]

,Player,Team,Positions,IP,K,W,SV,ERA,WHIP,ER,...,ERA+,WHIP+,W-L,ERAS,WHIPS,KS,SVS,WLS,SCORE,RANK
0,Max Scherzer,WSH,SP,207.6,269.0,16.0,0.0,2.904624,1.001927,67.0,...,300.477203,77.734893,8.0,94.0,100.0,100.0,0.0,93.0,387.0,1.5
1,Chris Sale,BOS,SP,184.6,248.0,15.0,0.0,2.632719,0.969664,54.0,...,317.380981,75.078329,9.0,97.0,98.0,92.0,0.0,100.0,387.0,1.5
2,Jacob deGrom,NYM,SP,207.8,248.0,15.0,0.0,2.728585,1.049086,63.0,...,337.347605,68.010167,7.0,100.0,93.0,92.0,0.0,86.0,371.0,3.0
3,Justin Verlander,HOU,SP,200.8,244.0,15.0,0.0,3.137450,1.045817,70.0,...,243.883538,66.375561,7.0,85.0,92.0,90.0,0.0,86.0,353.0,4.0
4,Corey Kluber,CLE,SP,207.1,220.0,16.0,0.0,3.215838,1.071946,74.0,...,235.301198,63.046707,8.0,84.0,90.0,80.0,0.0,93.0,347.0,5.0


## Combining Pitchers and Batters

Since we score across five pitching and six hitting categories, with the weekly winners determined by the highest aggregate scores in each category, it makes sense to compare score weight of batters vs. pitchers 

For the last couple of seasons, I've tried to "break the game" by focusing only on all the pitching categories plus two batting categories (AVG and BBKO) where there are batters generally available with high performance at a low cost. This has had mixed results, as the league adjusted accordingly.

In [634]:
batters_lim = batters[ ['Player', 'Team', 'Positions', 'AVGS', 'BBKOS', 'HRS', 'RS', 'RBIS', 'SBS', 'SCORE'] ]
pitchers_lim = pitchers[ ['Player', 'Team', 'Positions', 'ERAS', 'WHIPS', 'KS', 'SVS', 'WLS', 'SCORE'] ]
all_players = pd.concat([batters_lim, pitchers_lim],sort=True)
all_players['RANK'] = all_players['SCORE'].rank(method='average', na_option='keep', ascending=False, pct=False)
all_players = all_players.sort_values(['SCORE'], ascending=[0])

In [635]:
all_players.head(25)

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SVS,Team,WHIPS,WLS,RANK
0,85.0,100.0,NaN,90.0,NaN,Mike Trout,"CF,DH",81.0,96.0,52.0,504.0,NaN,LAA,NaN,NaN,1.0
1,94.0,77.0,NaN,68.0,NaN,Mookie Betts,"CF,RF",77.0,100.0,59.0,475.0,NaN,BOS,NaN,NaN,2.0
2,72.0,85.0,NaN,68.0,NaN,Jose Ramirez,"2B,3B",80.0,84.0,54.0,443.0,NaN,CLE,NaN,NaN,3.0
4,87.0,50.0,NaN,93.0,NaN,J.D. Martinez,"LF,RF,DH",100.0,81.0,9.0,420.0,NaN,BOS,NaN,NaN,4.0
3,79.0,61.0,NaN,85.0,NaN,Nolan Arenado,3B,97.0,82.0,4.0,408.0,NaN,COL,NaN,NaN,5.5
9,53.0,88.0,NaN,78.0,NaN,Bryce Harper,"CF,RF",83.0,82.0,24.0,408.0,NaN,PHI,NaN,NaN,5.5
5,84.0,61.0,NaN,61.0,NaN,Christian Yelich,"LF,CF,RF",77.0,84.0,37.0,404.0,NaN,MIL,NaN,NaN,7.0
8,70.0,65.0,NaN,66.0,NaN,Francisco Lindor,SS,70.0,84.0,41.0,396.0,NaN,CLE,NaN,NaN,8.5
11,70.0,80.0,NaN,63.0,NaN,Alex Bregman,"3B,SS",78.0,81.0,24.0,396.0,NaN,HOU,NaN,NaN,8.5
10,100.0,64.0,NaN,39.0,NaN,Jose Altuve,2B,63.0,78.0,48.0,392.0,NaN,HOU,NaN,NaN,10.0


## Join on list of currently drafted players

In [636]:
draft = pd.read_csv('./results.csv', header=1)
draft.head()

,Pick,Team,Player,Elapsed Time,Unnamed: 4
0,1,MitchMoreland'sBattingAverage,Mike Trout CF | LAA,NaN,NaN
1,2,Tommy's Shinebox,Mookie Betts RF | BOS,8 hrs 29 min 27 sec,NaN
2,3,La Flama Blanca,Jose Altuve 2B | HOU,32 min 24 sec,NaN
3,4,Low hanging fruit,Max Scherzer SP | WAS,2 min 59 sec,NaN
4,5,Foul_Tip,Nolan Arenado 3B | COL,57 min 17 sec,NaN


In [637]:
draft = draft[ ['Player', 'Team'] ]
draft['Name'] = draft['Player']
draft['Owner'] = draft['Team']
draft = draft[ draft['Player'] != 'Player' ]
draft['Player'] = draft['Name'].str.extract(r'^(.+ .+) \w+ \|', expand=False)
draft['Team'] = draft['Name'].str.extract(r'\|\s(\w\w\w|\w\w)(?:\s|$)', expand=False)
draft = draft.dropna()

In [638]:
draft = draft.replace('WAS', 'WSH')
draft = draft.replace('CHW', 'CWS')

In [639]:
draft.head()

,Player,Team,Name,Owner
0,Mike Trout,LAA,Mike Trout CF | LAA,MitchMoreland'sBattingAverage
1,Mookie Betts,BOS,Mookie Betts RF | BOS,Tommy's Shinebox
2,Jose Altuve,HOU,Jose Altuve 2B | HOU,La Flama Blanca
3,Max Scherzer,WSH,Max Scherzer SP | WAS,Low hanging fruit
4,Nolan Arenado,COL,Nolan Arenado 3B | COL,Foul_Tip


In [640]:
all_players_owners = all_players.merge(draft, on=['Player', 'Team'], how='outer')

In [641]:
all_players_owners.head()

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SVS,Team,WHIPS,WLS,RANK,Name,Owner
0,85.0,100.0,NaN,90.0,NaN,Mike Trout,"CF,DH",81.0,96.0,52.0,504.0,NaN,LAA,NaN,NaN,1.0,Mike Trout CF | LAA,MitchMoreland'sBattingAverage
1,94.0,77.0,NaN,68.0,NaN,Mookie Betts,"CF,RF",77.0,100.0,59.0,475.0,NaN,BOS,NaN,NaN,2.0,Mookie Betts RF | BOS,Tommy's Shinebox
2,72.0,85.0,NaN,68.0,NaN,Jose Ramirez,"2B,3B",80.0,84.0,54.0,443.0,NaN,CLE,NaN,NaN,3.0,Jose Ramirez 3B | CLE (Keeper),BC-Mizusawa South Monkeyz
3,87.0,50.0,NaN,93.0,NaN,J.D. Martinez,"LF,RF,DH",100.0,81.0,9.0,420.0,NaN,BOS,NaN,NaN,4.0,J.D. Martinez DH | BOS,stroke me strokeme
4,79.0,61.0,NaN,85.0,NaN,Nolan Arenado,3B,97.0,82.0,4.0,408.0,NaN,COL,NaN,NaN,5.5,Nolan Arenado 3B | COL,Foul_Tip


Ensure all players merged. Cross fingers that there aren't any players on the same team with the same name. That would be weird though. This takes advatage of the characterisitics of an outer join.

In [642]:
all_players_owners[all_players_owners['Player'].isnull()].head()

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SVS,Team,WHIPS,WLS,RANK,Name,Owner


In [643]:
all_players_owners[all_players_owners['Positions'].isnull()].head()

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SVS,Team,WHIPS,WLS,RANK,Name,Owner


## All Currently Undrafted

In [644]:
all_players_owners = all_players_owners[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS', 'Owner'] ]
free_agents = all_players_owners[all_players_owners['Owner'].isnull()]
del free_agents['Owner']
free_agents.head(50)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,SVS
48,Andrew McCutchen,"LF,RF",PHI,318.0,50.0,45.0,66.0,56.0,62.0,65.0,24.0,NaN,NaN,NaN,NaN,NaN
53,Vladimir Guerrero Jr.,3B,TOR,314.0,54.0,77.0,58.0,49.0,64.0,55.0,11.0,NaN,NaN,NaN,NaN,NaN
54,Eddie Rosario,LF,MIN,313.0,55.0,66.0,37.0,56.0,67.0,67.0,20.0,NaN,NaN,NaN,NaN,NaN
67,Mike Moustakas,"3B,DH",MIL,303.0,68.5,46.0,50.0,71.0,75.0,54.0,7.0,NaN,NaN,NaN,NaN,NaN
68,Justin Upton,"LF,DH",LAA,303.0,68.5,38.0,42.0,68.0,74.0,64.0,17.0,NaN,NaN,NaN,NaN,NaN
70,Joey Gallo,"1B,LF,CF,RF",TEX,301.0,70.5,11.0,42.0,93.0,77.0,67.0,11.0,NaN,NaN,NaN,NaN,NaN
71,Matt Olson,1B,OAK,300.0,72.0,31.0,52.0,73.0,75.0,65.0,4.0,NaN,NaN,NaN,NaN,NaN
72,Corey Seager,SS,LAD,299.0,73.0,69.0,54.0,46.0,59.0,64.0,7.0,NaN,NaN,NaN,NaN,NaN
75,David Peralta,LF,ARI,297.0,76.5,69.0,47.0,51.0,60.0,59.0,11.0,NaN,NaN,NaN,NaN,NaN
78,Eric Hosmer,1B,SD,296.0,78.5,57.0,56.0,46.0,64.0,60.0,13.0,NaN,NaN,NaN,NaN,NaN


## Positions

### 1B

In [645]:
first_base = all_players_owners[all_players_owners['Positions'].str.contains('1B')]
first_base = first_base[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [646]:
first_base.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
11,Paul Goldschmidt,1B,STL,388.0,12.5,70.0,69.0,68.0,78.0,79.0,24.0,Martin Awaiting Klub
15,Freddie Freeman,1B,ATL,379.0,16.0,84.0,66.0,61.0,77.0,74.0,17.0,Walk Off
17,Anthony Rizzo,1B,CHC,377.0,18.5,66.0,78.0,66.0,83.0,69.0,15.0,Low hanging fruit
25,Rhys Hoskins,"1B,LF",PHI,358.0,26.0,37.0,66.0,83.0,87.0,74.0,11.0,La Flama Blanca
26,Joey Votto,1B,CIN,355.0,27.0,71.0,97.0,49.0,65.0,66.0,7.0,Rakuten Golden Eagles
31,Cody Bellinger,"1B,CF",LAD,343.0,32.0,47.0,54.0,71.0,76.0,69.0,26.0,Foul_Tip
37,Whit Merrifield,"1B,2B,CF,RF,DH",KC,334.0,38.5,71.0,50.0,27.0,48.0,64.0,74.0,Regular Rubens
39,Matt Carpenter,"1B,2B,3B",STL,330.0,40.0,42.0,77.0,63.0,62.0,79.0,7.0,Regular Rubens
43,Daniel Murphy,"1B,2B",COL,322.0,44.5,86.0,56.0,46.0,65.0,60.0,9.0,Martin Awaiting Klub
47,Carlos Santana,"1B,3B",CLE,319.0,48.0,36.0,93.0,51.0,68.0,64.0,7.0,Tommy's Shinebox


### 2B

In [647]:
second_base = all_players_owners[all_players_owners['Positions'].str.contains('2B')]
second_base = second_base[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [648]:
second_base.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
2,Jose Ramirez,"2B,3B",CLE,443.0,3.0,72.0,85.0,68.0,80.0,84.0,54.0,BC-Mizusawa South Monkeyz
9,Jose Altuve,2B,HOU,392.0,10.0,100.0,64.0,39.0,63.0,78.0,48.0,La Flama Blanca
32,Javier Baez,"2B,3B,SS",CHC,340.0,33.0,63.0,22.0,68.0,81.0,69.0,37.0,Team Gorgeous
37,Whit Merrifield,"1B,2B,CF,RF,DH",KC,334.0,38.5,71.0,50.0,27.0,48.0,64.0,74.0,Regular Rubens
39,Matt Carpenter,"1B,2B,3B",STL,330.0,40.0,42.0,77.0,63.0,62.0,79.0,7.0,Regular Rubens
43,Daniel Murphy,"1B,2B",COL,322.0,44.5,86.0,56.0,46.0,65.0,60.0,9.0,Martin Awaiting Klub
58,Travis Shaw,"1B,3B,2B",MIL,310.0,59.0,41.0,59.0,66.0,73.0,58.0,13.0,Team Gorgeous
59,Adalberto Mondesi,"2B,SS",KC,310.0,59.0,38.0,23.0,44.0,52.0,57.0,96.0,Foul_Tip
63,Ozzie Albies,2B,ATL,308.0,63.5,57.0,44.0,44.0,58.0,70.0,35.0,Walk Off
76,Gleyber Torres,"2B,SS",NYY,297.0,76.5,53.0,44.0,56.0,70.0,54.0,20.0,MitchMoreland'sBattingAverage


### SS

In [649]:
shortstop = all_players_owners[all_players_owners['Positions'].str.contains('SS')]
shortstop = shortstop[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [650]:
shortstop.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
7,Francisco Lindor,SS,CLE,396.0,8.5,70.0,65.0,66.0,70.0,84.0,41.0,Walk Off
8,Alex Bregman,"3B,SS",HOU,396.0,8.5,70.0,80.0,63.0,78.0,81.0,24.0,Martin Awaiting Klub
10,Trea Turner,SS,WSH,391.0,11.0,68.0,51.0,39.0,54.0,79.0,100.0,Martin Awaiting Klub
18,Manny Machado,"3B,SS",SD,377.0,18.5,71.0,60.0,76.0,80.0,68.0,22.0,MitchMoreland'sBattingAverage
24,Trevor Story,SS,COL,359.0,25.0,59.0,32.0,76.0,82.0,67.0,43.0,Team Gorgeous
32,Javier Baez,"2B,3B,SS",CHC,340.0,33.0,63.0,22.0,68.0,81.0,69.0,37.0,Team Gorgeous
33,Xander Bogaerts,SS,BOS,339.0,34.0,72.0,55.0,44.0,75.0,69.0,24.0,BC-Mizusawa South Monkeyz
46,Carlos Correa,SS,HOU,320.0,47.0,56.0,59.0,54.0,75.0,65.0,11.0,BC-Mizusawa South Monkeyz
49,Jean Segura,SS,PHI,318.0,50.0,76.0,48.0,27.0,49.0,70.0,48.0,Low hanging fruit
59,Adalberto Mondesi,"2B,SS",KC,310.0,59.0,38.0,23.0,44.0,52.0,57.0,96.0,Foul_Tip


### 3B

In [651]:
third_base = all_players_owners[all_players_owners['Positions'].str.contains('3B')]
third_base = third_base[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [652]:
third_base.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
2,Jose Ramirez,"2B,3B",CLE,443.0,3.0,72.0,85.0,68.0,80.0,84.0,54.0,BC-Mizusawa South Monkeyz
4,Nolan Arenado,3B,COL,408.0,5.5,79.0,61.0,85.0,97.0,82.0,4.0,Foul_Tip
8,Alex Bregman,"3B,SS",HOU,396.0,8.5,70.0,80.0,63.0,78.0,81.0,24.0,Martin Awaiting Klub
18,Manny Machado,"3B,SS",SD,377.0,18.5,71.0,60.0,76.0,80.0,68.0,22.0,MitchMoreland'sBattingAverage
28,Anthony Rendon,3B,WSH,352.0,29.0,75.0,69.0,54.0,75.0,68.0,11.0,stroke me strokeme
29,Kris Bryant,"3B,RF",CHC,349.0,30.0,63.0,62.0,63.0,69.0,79.0,13.0,Rakuten Golden Eagles
32,Javier Baez,"2B,3B,SS",CHC,340.0,33.0,63.0,22.0,68.0,81.0,69.0,37.0,Team Gorgeous
39,Matt Carpenter,"1B,2B,3B",STL,330.0,40.0,42.0,77.0,63.0,62.0,79.0,7.0,Regular Rubens
45,Eugenio Suarez,3B,CIN,321.0,46.0,53.0,53.0,68.0,77.0,63.0,7.0,Low hanging fruit
47,Carlos Santana,"1B,3B",CLE,319.0,48.0,36.0,93.0,51.0,68.0,64.0,7.0,Tommy's Shinebox


### C

In [653]:
catcher = all_players_owners[all_players_owners['Positions'].str.contains('^C$|C,', regex=True)]
catcher = catcher[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [654]:
catcher.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
105,J.T. Realmuto,"C,1B",PHI,277.0,106.5,59.0,43.0,49.0,61.0,54.0,11.0,Team Gorgeous
111,Gary Sanchez,"C,DH",NYY,273.0,111.5,34.0,45.0,68.0,67.0,55.0,4.0,La Flama Blanca
177,Buster Posey,"C,1B",SF,243.0,178.5,68.0,66.0,20.0,42.0,40.0,7.0,NaN
211,Yadier Molina,C,STL,231.0,212.5,52.0,47.0,34.0,51.0,36.0,11.0,NaN
213,Yasmani Grandal,C,MIL,230.0,217.0,33.0,57.0,49.0,49.0,40.0,2.0,Martin Awaiting Klub
240,Willson Contreras,C,CHC,220.0,240.0,44.0,51.0,32.0,47.0,37.0,9.0,Foul_Tip
295,Wilson Ramos,"C,DH",NYM,202.0,294.0,53.0,45.0,34.0,46.0,24.0,0.0,BC-Mizusawa South Monkeyz
341,Francisco Cervelli,C,PIT,188.0,341.5,44.0,57.0,20.0,33.0,27.0,7.0,NaN
351,Danny Jansen,C,TOR,184.0,353.0,39.0,53.0,24.0,32.0,29.0,7.0,NaN
396,Tucker Barnhart,C,CIN,173.0,398.0,39.0,56.0,17.0,31.0,26.0,4.0,NaN


### OF

In [655]:
outfield = all_players_owners[all_players_owners['Positions'].str.contains('OF|LF|RF|CF', regex=True)]
outfield = outfield[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [656]:
outfield[:24]

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
0,Mike Trout,"CF,DH",LAA,504.0,1.0,85.0,100.0,90.0,81.0,96.0,52.0,MitchMoreland'sBattingAverage
1,Mookie Betts,"CF,RF",BOS,475.0,2.0,94.0,77.0,68.0,77.0,100.0,59.0,Tommy's Shinebox
3,J.D. Martinez,"LF,RF,DH",BOS,420.0,4.0,87.0,50.0,93.0,100.0,81.0,9.0,stroke me strokeme
5,Bryce Harper,"CF,RF",PHI,408.0,5.5,53.0,88.0,78.0,83.0,82.0,24.0,Regular Rubens
6,Christian Yelich,"LF,CF,RF",MIL,404.0,7.0,84.0,61.0,61.0,77.0,84.0,37.0,BC-Mizusawa South Monkeyz
12,Juan Soto,LF,WSH,388.0,12.5,73.0,79.0,66.0,78.0,77.0,15.0,Rakuten Golden Eagles
16,Giancarlo Stanton,"LF,RF,DH",NYY,378.0,17.0,54.0,44.0,100.0,93.0,80.0,7.0,Tommy's Shinebox
19,Aaron Judge,"RF,DH",NYY,376.0,20.0,51.0,62.0,88.0,78.0,82.0,15.0,Foul_Tip
20,Ronald Acuna,"LF,CF",ATL,375.0,21.0,65.0,42.0,68.0,71.0,79.0,50.0,Team Gorgeous
21,Charlie Blackmon,CF,COL,372.0,22.5,81.0,51.0,63.0,62.0,89.0,26.0,La Flama Blanca


In [657]:
outfield[25:49]

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
55,Michael Brantley,"LF,DH",HOU,311.0,56.5,74.0,63.0,34.0,58.0,58.0,24.0,Regular Rubens
60,Mitch Haniger,"CF,RF",SEA,309.0,61.5,56.0,51.0,54.0,68.0,65.0,15.0,Walk Off
64,Nicholas Castellanos,RF,DET,307.0,65.5,69.0,39.0,56.0,74.0,62.0,7.0,MitchMoreland'sBattingAverage
68,Justin Upton,"LF,DH",LAA,303.0,68.5,38.0,42.0,68.0,74.0,64.0,17.0,NaN
69,Michael Conforto,"LF,CF,RF",NYM,301.0,70.5,36.0,59.0,66.0,70.0,63.0,7.0,BC-Mizusawa South Monkeyz
70,Joey Gallo,"1B,LF,CF,RF",TEX,301.0,70.5,11.0,42.0,93.0,77.0,67.0,11.0,NaN
75,David Peralta,LF,ARI,297.0,76.5,69.0,47.0,51.0,60.0,59.0,11.0,NaN
80,A.J. Pollock,CF,LAD,295.0,80.5,48.0,46.0,49.0,56.0,59.0,37.0,NaN
85,Ender Inciarte,CF,ATL,292.0,86.0,64.0,56.0,20.0,42.0,60.0,50.0,NaN
88,David Dahl,"LF,CF,RF",COL,289.0,88.5,58.0,33.0,56.0,65.0,53.0,24.0,NaN


In [658]:
#outfield[50:74]

### SP

In [659]:
starter = all_players_owners[all_players_owners['Positions'].str.contains('SP')]
starter = starter[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS', 'Owner'] ]

In [660]:
starter[25:49]

,Player,Positions,Team,SCORE,RANK,ERAS,WHIPS,KS,WLS,SVS,Owner
156,Jose Berrios,SP,MIN,250.0,158.0,60.0,64.0,69.0,57.0,0.0,Martin Awaiting Klub
157,Masahiro Tanaka,SP,NYY,250.0,158.0,59.0,69.0,58.0,64.0,0.0,NaN
161,Mike Foltynewicz,SP,ATL,248.0,161.5,64.0,62.0,65.0,57.0,0.0,NaN
165,J.A. Happ,SP,NYY,246.0,166.0,60.0,63.0,59.0,64.0,0.0,NaN
171,Chris Archer,SP,PIT,244.0,173.0,63.0,59.0,72.0,50.0,0.0,NaN
172,Jameson Taillon,SP,PIT,244.0,173.0,67.0,65.0,62.0,50.0,0.0,Martin Awaiting Klub
175,Robbie Ray,SP,ARI,244.0,173.0,63.0,55.0,76.0,50.0,0.0,NaN
179,Zack Wheeler,SP,NYM,243.0,178.5,68.0,63.0,62.0,50.0,0.0,NaN
180,Rich Hill,SP,LAD,242.0,183.0,65.0,67.0,53.0,57.0,0.0,NaN
182,German Marquez,SP,COL,242.0,183.0,60.0,60.0,72.0,50.0,0.0,Regular Rubens


### RP

In [661]:
relief = all_players_owners[all_players_owners['Positions'].str.contains('RP')]
relief = relief[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS', 'Owner'] ]

In [662]:
relief[:24]

,Player,Positions,Team,SCORE,RANK,ERAS,WHIPS,KS,WLS,SVS,Owner
74,Edwin Diaz,RP,NYM,298.0,74.5,67.0,65.0,33.0,36.0,97.0,Foul_Tip
92,Brad Hand,RP,CLE,288.0,91.5,61.0,58.0,29.0,43.0,97.0,NaN
96,Aroldis Chapman,RP,NYY,284.0,97.5,63.0,57.0,29.0,43.0,92.0,MitchMoreland'sBattingAverage
97,Kenley Jansen,RP,LAD,284.0,97.5,61.0,63.0,24.0,36.0,100.0,NaN
99,Blake Treinen,RP,OAK,280.0,100.5,67.0,58.0,25.0,43.0,87.0,Foul_Tip
119,Craig Kimbrel,RP,NaN,270.0,120.5,63.0,59.0,29.0,43.0,76.0,NaN
120,Felipe Vazquez,RP,PIT,270.0,120.5,64.0,56.0,25.0,36.0,89.0,NaN
123,Roberto Osuna,RP,HOU,268.0,125.0,59.0,60.0,18.0,36.0,95.0,Low hanging fruit
125,Sean Doolittle,RP,WSH,268.0,125.0,62.0,63.0,21.0,43.0,79.0,NaN
143,Kirby Yates,RP,SD,257.0,143.5,58.0,57.0,27.0,36.0,79.0,NaN


## My Team

In [663]:
my_team = all_players_owners[ all_players_owners['Owner'] == 'Tommy\'s Shinebox' ]
my_team

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,SVS,Owner
1,Mookie Betts,"CF,RF",BOS,475.0,2.0,94.0,77.0,68.0,77.0,100.0,59.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
16,Giancarlo Stanton,"LF,RF,DH",NYY,378.0,17.0,54.0,44.0,100.0,93.0,80.0,7.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
30,Corey Kluber,SP,CLE,347.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,84.0,90.0,80.0,93.0,0.0,Tommy's Shinebox
34,Nelson Cruz,DH,MIN,338.0,35.5,58.0,50.0,80.0,85.0,63.0,2.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
41,Lorenzo Cain,CF,MIL,324.0,41.5,75.0,62.0,27.0,38.0,68.0,54.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
47,Carlos Santana,"1B,3B",CLE,319.0,48.0,36.0,93.0,51.0,68.0,64.0,7.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
94,Patrick Corbin,SP,WSH,286.0,94.5,NaN,NaN,NaN,NaN,NaN,NaN,74.0,69.0,72.0,71.0,0.0,Tommy's Shinebox
126,Charlie Morton,SP,TB,267.0,127.0,NaN,NaN,NaN,NaN,NaN,NaN,70.0,64.0,62.0,71.0,0.0,Tommy's Shinebox
208,Jose Martinez,"1B,RF",STL,232.0,209.5,66.0,51.0,29.0,45.0,37.0,4.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox


In [664]:
round(my_team[ ['AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS'] ].mean())

AVGS     64.0
BBKOS    63.0
HRS      59.0
RBIS     68.0
RS       69.0
SBS      22.0
ERAS     76.0
WHIPS    74.0
KS       71.0
WLS      78.0
SVS       0.0
dtype: float64

## Roto Analysis

In [665]:
cols = ['Team', 'Count', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS' ]
roto_score = pd.DataFrame(columns = cols)
for team in pd.unique(all_players_owners['Owner'].dropna()):
    roster = all_players_owners[ all_players_owners['Owner'] == team ]
    row = []
    row.append(team)
    row.append(len(roster))
    for stat in ['AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS']:
        if roster[stat].isnull().all():
            row.append(0)
        else:
            row.append(round(roster[stat].mean()))
    row = tuple(row)
    rosterdf = pd.DataFrame([row], columns=cols)
    roto_score = roto_score.append(rosterdf, ignore_index=True)

In [666]:
roto_score

,Team,Count,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,SVS
0,MitchMoreland'sBattingAverage,8,63,61,68,73,67,22,73,70,50,59,31
1,Tommy's Shinebox,9,64,63,59,68,69,22,76,74,71,78,0
2,BC-Mizusawa South Monkeyz,8,60,59,52,69,66,27,0,0,0,0,0
3,stroke me strokeme,7,67,49,73,84,68,6,82,86,84,86,0
4,Foul_Tip,9,55,49,53,63,63,33,67,62,29,40,92
5,Regular Rubens,9,59,64,53,65,66,23,71,66,73,64,0
6,Walk Off,8,67,53,53,65,71,36,76,71,71,69,0
7,Martin Awaiting Klub,9,63,62,54,64,70,29,65,67,66,55,0
8,La Flama Blanca,6,60,59,63,68,71,20,70,70,66,64,0
9,Rakuten Golden Eagles,6,65,73,57,66,73,18,88,84,86,90,0


In [667]:
cols = ['AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SVS' ]
for col in cols:
    roto_score[col] = roto_score[col].rank(method='average', na_option='keep', ascending=True, pct=False)
roto_score['Score'] = roto_score[cols].sum(axis=1)

In [668]:
roto_score.sort_values(['Score'], ascending=False)

,Team,Count,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,SVS,Score
9,Rakuten Golden Eagles,6,10.0,12.0,7.0,5.0,12.0,2.0,12.0,11.0,12.0,12.0,5.0,100.0
3,stroke me strokeme,7,11.5,2.5,12.0,12.0,7.0,1.0,10.5,12.0,11.0,11.0,5.0,95.5
1,Tommy's Shinebox,9,9.0,10.0,8.0,7.5,8.0,5.0,7.5,8.0,7.5,9.0,5.0,84.5
11,Team Gorgeous,8,2.0,1.0,10.0,11.0,3.0,10.0,10.5,10.0,10.0,10.0,5.0,82.5
6,Walk Off,8,11.5,4.0,3.0,3.5,10.5,12.0,7.5,7.0,7.5,7.5,5.0,79.0
0,MitchMoreland'sBattingAverage,8,7.5,8.0,11.0,10.0,6.0,5.0,6.0,5.5,3.0,4.0,10.0,76.0
10,Low hanging fruit,9,5.0,5.0,6.0,6.0,2.0,5.0,9.0,9.0,6.0,7.5,11.0,71.5
8,La Flama Blanca,6,5.0,6.5,9.0,7.5,10.5,3.0,4.0,5.5,4.5,5.5,5.0,66.0
7,Martin Awaiting Klub,9,7.5,9.0,5.0,2.0,9.0,9.0,2.0,4.0,4.5,3.0,5.0,60.0
5,Regular Rubens,9,3.0,11.0,3.0,3.5,4.5,7.0,5.0,3.0,9.0,5.5,5.0,59.5


In [669]:
roto_score_adj = roto_score
del roto_score_adj['SVS']
cols = ['AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS' ]
roto_score_adj['Score'] = roto_score[cols].sum(axis=1)

In [670]:
roto_score_adj.sort_values(['Score'], ascending=False)

,Team,Count,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,Score
9,Rakuten Golden Eagles,6,10.0,12.0,7.0,5.0,12.0,2.0,12.0,11.0,12.0,12.0,95.0
3,stroke me strokeme,7,11.5,2.5,12.0,12.0,7.0,1.0,10.5,12.0,11.0,11.0,90.5
1,Tommy's Shinebox,9,9.0,10.0,8.0,7.5,8.0,5.0,7.5,8.0,7.5,9.0,79.5
11,Team Gorgeous,8,2.0,1.0,10.0,11.0,3.0,10.0,10.5,10.0,10.0,10.0,77.5
6,Walk Off,8,11.5,4.0,3.0,3.5,10.5,12.0,7.5,7.0,7.5,7.5,74.0
0,MitchMoreland'sBattingAverage,8,7.5,8.0,11.0,10.0,6.0,5.0,6.0,5.5,3.0,4.0,66.0
8,La Flama Blanca,6,5.0,6.5,9.0,7.5,10.5,3.0,4.0,5.5,4.5,5.5,61.0
10,Low hanging fruit,9,5.0,5.0,6.0,6.0,2.0,5.0,9.0,9.0,6.0,7.5,60.5
7,Martin Awaiting Klub,9,7.5,9.0,5.0,2.0,9.0,9.0,2.0,4.0,4.5,3.0,55.0
5,Regular Rubens,9,3.0,11.0,3.0,3.5,4.5,7.0,5.0,3.0,9.0,5.5,54.5
